In [1]:
import gadgets
import datasets
import transformers
import pandas as pd
import os

[2023-10-24 10:48:15,683] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [12]:
ds = datasets.load_dataset("gsm8k", "main")
print("ORIGINAL SPLIT:")
print(ds)
seed = 0
ds_train = ds["train"]
ds_test = ds["test"]
ds_train, ds_valid = ds_train.train_test_split(seed=seed, test_size=200).values()
ds = datasets.DatasetDict({"train": ds_train, "validation": ds_valid, "test": ds_test})
print("NEW SPLIT:")
print(ds)

ORIGINAL SPLIT:
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})
NEW SPLIT:
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7273
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 200
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [13]:
calc = gadgets.gadget.Calculator()

os.makedirs("../data/gsm8k", exist_ok=True)
df = {}
for split in ["train", "test", "validation"]:
    df[split] = ds[split].to_pandas()
    examples = df[split].apply(gadgets.gsm8k.parse, axis=1)
    df[split].drop(columns=["answer"], inplace=True)
    df[split]["chain"] = examples.apply(lambda x: gadgets.markup.to_model_markup(example=x)).apply(str)
    df[split]["result"] = examples.apply(lambda x: x.result)
    df[split]["result_float"] = df[split]["result"].apply(calc._float_eval)
    df[split].to_json(f"../data/gsm8k/{split}.jsonl", orient="records", lines=True, force_ascii=False)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")


In [14]:
ds = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df["train"]),
    "validation": datasets.Dataset.from_pandas(df["validation"]),
    "test": datasets.Dataset.from_pandas(df["test"])
})
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 7273
    })
    validation: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 200
    })
    test: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 1319
    })
})

In [15]:
ds.push_to_hub("MU-NLPC/Calc-gsm8k")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
with open("../data/gsm8k/train.jsonl") as f:
    content = f.read()
with open("../data/gsm8k/test.jsonl") as f:
    content += f.read()
content.count('\u2028')

0

In [7]:
foo = pd.Series([x for x in content if not x.isascii()]).value_counts().reset_index()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("t5-base")
# TODO many random unicode characters that might not be in the tokenizer's vocab

In [ ]:
foo["tokenized"] = foo["index"].apply(tokenizer.encode).apply(tokenizer.decode, skip_special_tokens=True)

In [ ]:
foo